In [1]:
import pandas as pd
import math
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
from collections import defaultdict
from scipy.stats.stats import pearsonr
import seaborn as sn

In [2]:
df = pd.read_csv('dataset/matches_datacleaning.csv', index_col = 0)
pd.set_option('display.max_columns', None) # to visualize all the columns
df.info()


/Users/domenico/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


<class 'pandas.core.frame.DataFrame'>
Int64Index: 184890 entries, 0 to 186072
Data columns (total 51 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   tourney_id          184890 non-null  object 
 1   tourney_name        184890 non-null  object 
 2   surface             184728 non-null  object 
 3   draw_size           184890 non-null  float64
 4   tourney_level       184890 non-null  object 
 5   tourney_date        184890 non-null  object 
 6   match_num           184890 non-null  float64
 7   winner_id           184890 non-null  float64
 8   winner_entry        25701 non-null   object 
 9   winner_name         184890 non-null  object 
 10  winner_hand         184890 non-null  object 
 11  winner_ht           48560 non-null   float64
 12  winner_ioc          184890 non-null  object 
 13  winner_age          182080 non-null  float64
 14  loser_id            184890 non-null  float64
 15  loser_entry         43927 non-null

## new database building

In [3]:
name = pd.concat([df['winner_name'], df['loser_name']]) 
#name.rename(columns={0: 'name'}, inplace = True)
name.drop_duplicates(inplace=True)


new_df = pd.DataFrame(name)
new_df.rename(columns={0: 'name'}, inplace = True)
print(new_df)

                      name
0            KEI NISHIKORI
1          DANIIL MEDVEDEV
3       JO-WILFRIED TSONGA
5            JEREMY CHARDY
8           ALEX DE MINAUR
...                    ...
185021    DILARA SULTANOVA
185560    MADELEINE JESSUP
185780          KATIE CODD
185781           AMY HUANG
185940     CHLOE HENDERSON

[10098 rows x 1 columns]


In [4]:
def create_new_colums(winner_attr, loser_attr, attr):
    col1=df[['winner_name',winner_attr]]
    col2=df[['loser_name',loser_attr]]
    col2.rename(columns={'loser_name': 'winner_name'}, inplace = True)
    col2.rename(columns={loser_attr: winner_attr}, inplace = True)
    name2 = col1.append(col2) 
    name2.rename(columns={'winner_name': 'name',winner_attr:attr}, inplace = True)
    name2.drop_duplicates(inplace=True)
    return name2


## num match of each player as winner and loser

In [5]:
num_match=df['winner_name'].value_counts(dropna=False)
num_match=pd.DataFrame(num_match)
num_match.reset_index(inplace=True)
num_match.rename(columns={'index': 'name','winner_name':'match_won'}, inplace = True)

new_df=pd.merge(new_df, num_match, how='left', on='name')
num_match.rename(columns={'match_won': 'num_match'}, inplace = True)

num_match1=df['loser_name'].value_counts(dropna=False)
num_match1=pd.DataFrame(num_match1)
num_match1.reset_index(inplace=True)
num_match1.rename(columns={'index': 'name','loser_name':'match_loss'}, inplace = True)

new_df=pd.merge(new_df, num_match1, how='left', on='name')
num_match1.rename(columns={'match_loss': 'num_match'}, inplace = True)

num_match=num_match1.append(num_match)
num_match=num_match.groupby(['name']).num_match.sum()

num_match=pd.DataFrame(num_match)
num_match.reset_index(inplace=True)

new_df=pd.merge(new_df, num_match, how='left', on='name')
new_df['match_won']=new_df['match_won'].fillna(0)
new_df['match_loss']=new_df['match_loss'].fillna(0)
print(new_df)

                     name  match_won  match_loss  num_match
0           KEI NISHIKORI      188.0        92.0        280
1         DANIIL MEDVEDEV      263.0       113.0        376
2      JO-WILFRIED TSONGA      123.0        71.0        194
3           JEREMY CHARDY      148.0       130.0        278
4          ALEX DE MINAUR      160.0       107.0        267
...                   ...        ...         ...        ...
10093    DILARA SULTANOVA        0.0         1.0          1
10094    MADELEINE JESSUP        0.0         1.0          1
10095          KATIE CODD        0.0         1.0          1
10096           AMY HUANG        0.0         1.0          1
10097     CHLOE HENDERSON        0.0         1.0          1

[10098 rows x 4 columns]


In [6]:
new_df=new_df[new_df['num_match']>2]
print(new_df)

                     name  match_won  match_loss  num_match
0           KEI NISHIKORI      188.0        92.0        280
1         DANIIL MEDVEDEV      263.0       113.0        376
2      JO-WILFRIED TSONGA      123.0        71.0        194
3           JEREMY CHARDY      148.0       130.0        278
4          ALEX DE MINAUR      160.0       107.0        267
...                   ...        ...         ...        ...
10030    MADALENA MARQUES        0.0         3.0          3
10055  GIULIA FINOCCHIARO        0.0         3.0          3
10056      ALICE BENVEGNU        0.0         3.0          3
10083  ELIF SANEM KARADAG        0.0         3.0          3
10090  ASLI YAGMUR KUNDUK        0.0         3.0          3

[6645 rows x 4 columns]


In [7]:
new_df['ratio']=new_df['match_won']/new_df['num_match']
print(new_df)


                     name  match_won  match_loss  num_match     ratio
0           KEI NISHIKORI      188.0        92.0        280  0.671429
1         DANIIL MEDVEDEV      263.0       113.0        376  0.699468
2      JO-WILFRIED TSONGA      123.0        71.0        194  0.634021
3           JEREMY CHARDY      148.0       130.0        278  0.532374
4          ALEX DE MINAUR      160.0       107.0        267  0.599251
...                   ...        ...         ...        ...       ...
10030    MADALENA MARQUES        0.0         3.0          3  0.000000
10055  GIULIA FINOCCHIARO        0.0         3.0          3  0.000000
10056      ALICE BENVEGNU        0.0         3.0          3  0.000000
10083  ELIF SANEM KARADAG        0.0         3.0          3  0.000000
10090  ASLI YAGMUR KUNDUK        0.0         3.0          3  0.000000

[6645 rows x 5 columns]


## Statistics of players during period

In [8]:
tourney_year=df['tourney_date'].str[:4].astype('int32')
df['tourney_year']=tourney_year

In [12]:
year=tourney_year.unique()
print(year)

#for i in year:
i=2016
df_year_winner=df[df['tourney_year']==i]['winner_name']
num_match_won=df_year_winner.value_counts(dropna=False)
num_match_won=pd.DataFrame(num_match_won)
num_match_won.reset_index(inplace=True)
num_match_won.rename(columns={'index': 'name','winner_name':'match_won_'+str(i)}, inplace = True)

#new_df=pd.merge(new_df, num_match, how='left', on='name')
num_match_won.rename(columns={'match_won_'+str(i): 'num_match_'+str(i)}, inplace = True)

df_year_loser=df[df['tourney_year']==i]['loser_name']
num_match_loss=df_year_loser.value_counts(dropna=False)
num_match_loss=pd.DataFrame(num_match_loss)
num_match_loss.reset_index(inplace=True)
num_match_loss.rename(columns={'index': 'name','loser_name':'match_loss_'+str(i)}, inplace = True)

#new_df=pd.merge(new_df, num_match1, how='left', on='name')
num_match_loss.rename(columns={'match_loss_'+str(i): 'num_match_'+str(i)}, inplace = True)
num_match=num_match_loss.append(num_match_won)

num_match=num_match.groupby(['name'])['num_match_'+str(i)].sum()
#num_match_won['ratio_'+str(i)]=num_match_won['match_won']/new_df['num_match']

print(num_match_won)
print(num_match_loss)
    #new_df['ratio']=new_df['match_won']/new_df['num_match']

    #num_match=pd.DataFrame(num_match)
    #num_match.reset_index(inplace=True)

    #new_df=pd.merge(new_df, num_match, how='left', on='name')

#print(new_df)
#print(df[df['tourney_year']==2016]['winner_name'])

[2018 2019 2017 2016 2020 2021]
name
AAREYALEE AMRUTSINH CHAVAN     1
AARON ADDISON                  6
AASTHA DARGUDE                 2
ABBIE MYERS                   33
ABDULHAMID MUBARAK             1
                              ..
ZORICA SPASOJEVIC              2
ZOZIYA KARDAVA                 2
ZUZANA LUKNAROVA               2
ZUZANA ZALABSKA                3
ZUZANA ZLOCHOVA               51
Name: num_match_2016, Length: 4754, dtype: int64


## Adding hand of players

In [6]:
value=create_new_colums('winner_hand','loser_hand','hand')
value.at[180480, 'hand'] = 'R'
value.drop_duplicates(inplace=True)

#andare a rivedere l'indice dalla tabella originaria

/Users/domenico/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [7]:
new_df=pd.merge(new_df, value, how='left', on='name')
print(new_df)

                     name  num_match hand
0           KEI NISHIKORI        280    R
1         DANIIL MEDVEDEV        376    R
2      JO-WILFRIED TSONGA        194    R
3           JEREMY CHARDY        278    R
4          ALEX DE MINAUR        267    R
...                   ...        ...  ...
10093    DILARA SULTANOVA          1    U
10094    MADELEINE JESSUP          1    U
10095          KATIE CODD          1    U
10096           AMY HUANG          1    U
10097     CHLOE HENDERSON          1    U

[10098 rows x 3 columns]


## adding sex of players

In [8]:
value=create_new_colums('winner_gender','loser_gender','gender')
print(value.gender)

/Users/domenico/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


0         M
1         M
3         M
5         M
8         M
         ..
185021    F
185560    F
185780    F
185781    F
185940    F
Name: gender, Length: 10098, dtype: object


In [9]:
#new_df['gender']=value.gender

new_df=pd.merge(new_df, value, how='left', on='name')
print(new_df)

                     name  num_match hand gender
0           KEI NISHIKORI        280    R      M
1         DANIIL MEDVEDEV        376    R      M
2      JO-WILFRIED TSONGA        194    R      M
3           JEREMY CHARDY        278    R      M
4          ALEX DE MINAUR        267    R      M
...                   ...        ...  ...    ...
10093    DILARA SULTANOVA          1    U      F
10094    MADELEINE JESSUP          1    U      F
10095          KATIE CODD          1    U      F
10096           AMY HUANG          1    U      F
10097     CHLOE HENDERSON          1    U      F

[10098 rows x 4 columns]


## ioc of players

In [10]:
value=create_new_colums('winner_ioc','loser_ioc','ioc')
value.drop_duplicates(inplace=True)

new_df=pd.merge(new_df, value, how='left', on='name')
print(new_df)

/Users/domenico/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


                     name  num_match hand gender  ioc
0           KEI NISHIKORI        280    R      M  JPN
1         DANIIL MEDVEDEV        376    R      M  RUS
2      JO-WILFRIED TSONGA        194    R      M  FRA
3           JEREMY CHARDY        278    R      M  FRA
4          ALEX DE MINAUR        267    R      M  AUS
...                   ...        ...  ...    ...  ...
10093    DILARA SULTANOVA          1    U      F  TKM
10094    MADELEINE JESSUP          1    U      F  USA
10095          KATIE CODD          1    U      F  USA
10096           AMY HUANG          1    U      F  USA
10097     CHLOE HENDERSON          1    U      F  USA

[10098 rows x 5 columns]


## ages of players

In [11]:

LoserId_name = df.groupby(['winner_name']).winner_age.nunique() 
LoserId_name = LoserId_name[LoserId_name > 1] 

df[df.winner_name.isin(LoserId_name.index)].groupby(['winner_name']).winner_age.unique()

winner_name
ABBIE MYERS                        [23.0, 22.0, 24.0, 25.0, 26.0]
ABHINAV SANJEEV SHANMUGAM                      [18.0, 19.0, 21.0]
ABIGAIL DESIATNIKOV                                  [17.0, 15.0]
ABIGAIL RENCHELI                                     [17.0, 18.0]
ABIGAIL TERE APISAH                [25.0, 24.0, 26.0, 27.0, 28.0]
                                             ...                 
ZOE KRUGER                   [15.0, 14.0, 16.0, 17.0, 19.0, 18.0]
ZOE RICHARD                              [19.0, 20.0, 21.0, 22.0]
ZOZIYA KARDAVA                     [16.0, 17.0, 18.0, 20.0, 19.0]
ZSOMBOR PIROS                            [18.0, 19.0, 22.0, 20.0]
ZUZANA ZLOCHOVA              [27.0, 28.0, 26.0, 30.0, 29.0, 31.0]
Name: winner_age, Length: 3643, dtype: object

In [12]:
gg=df

gg['winner_birth']=(gg['tourney_date'].str[:4].astype('int32')-gg['winner_age'])
gg['loser_birth']=(gg['tourney_date'].str[:4].astype('int32')-gg['loser_age'])
#print(var)
print()

In [13]:
winner_birth=gg.groupby(['winner_name']).winner_birth.min() #print(gg.groupby(['winner_name']).birth.min())
winner_birth = pd.DataFrame(winner_birth)
winner_birth.reset_index(inplace=True)
winner_birth.rename(columns={'winner_name': 'name', 'winner_birth': 'birth'}, inplace = True)

loser_birth=gg.groupby(['loser_name']).loser_birth.min() #print(gg.groupby(['loser_name']).birth.min())
loser_birth = pd.DataFrame(loser_birth)
loser_birth.reset_index(inplace=True)
loser_birth.rename(columns={'loser_name': 'name', 'loser_birth': 'birth'}, inplace = True)

name_birth=winner_birth.append(loser_birth)
name_birth.dropna(inplace=True)
name_birth.drop_duplicates(inplace=True)
name_birth=name_birth.groupby(name_birth['name']).birth.min()

name_birth = pd.DataFrame(name_birth)
name_birth.reset_index(inplace=True)
#print(name_birth)


In [14]:
new_df=pd.merge(new_df, name_birth, how='left', on='name')
print(new_df)

#controllare perchè l'anno non è intero

                     name  num_match hand gender  ioc   birth
0           KEI NISHIKORI        280    R      M  JPN  1980.5
1         DANIIL MEDVEDEV        376    R      M  RUS  1981.5
2      JO-WILFRIED TSONGA        194    R      M  FRA  1984.0
3           JEREMY CHARDY        278    R      M  FRA  1986.0
4          ALEX DE MINAUR        267    R      M  AUS  1998.0
...                   ...        ...  ...    ...  ...     ...
10093    DILARA SULTANOVA          1    U      F  TKM     NaN
10094    MADELEINE JESSUP          1    U      F  USA     NaN
10095          KATIE CODD          1    U      F  USA     NaN
10096           AMY HUANG          1    U      F  USA     NaN
10097     CHLOE HENDERSON          1    U      F  USA     NaN

[10098 rows x 6 columns]


## height of players

In [15]:
value=create_new_colums('winner_ht','loser_ht','ht')
value.dropna(inplace=True)
value.drop_duplicates(inplace=True)

name_ht = value.groupby(['name']).ht.max() 

new_df=pd.merge(new_df, name_ht, how='left', on='name')
print(new_df)

#completare aggiungendo l'altezza media tramite le nazionalità dei giocatori

                     name  num_match hand gender  ioc   birth     ht
0           KEI NISHIKORI        280    R      M  JPN  1980.5  178.0
1         DANIIL MEDVEDEV        376    R      M  RUS  1981.5  198.0
2      JO-WILFRIED TSONGA        194    R      M  FRA  1984.0  188.0
3           JEREMY CHARDY        278    R      M  FRA  1986.0  188.0
4          ALEX DE MINAUR        267    R      M  AUS  1998.0  183.0
...                   ...        ...  ...    ...  ...     ...    ...
10093    DILARA SULTANOVA          1    U      F  TKM     NaN    NaN
10094    MADELEINE JESSUP          1    U      F  USA     NaN    NaN
10095          KATIE CODD          1    U      F  USA     NaN    NaN
10096           AMY HUANG          1    U      F  USA     NaN    NaN
10097     CHLOE HENDERSON          1    U      F  USA     NaN    NaN

[10098 rows x 7 columns]


/Users/domenico/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
